# Description of all GVEC parameters

* We choose to use a dictionary of strings or array of strings for linebreaks. 
* If another parameter is mentioned, we use a string  `lnk_to_param(paramname)` or `lnk_to_param(linktext|paramname)` so that we can insert the link later. 
* the dictionary allows for (re-)formatting (either to markdown, with links, or to a screen output) and filtering the parameter list. 




In [ ]:
import yaml

dict = {}

# template:
# dict["paramname"] = {
#     "category": ["A","B"],
#     "subtitle": "[mandatory,essential,...] parameter"  # subtitle next to paramname
#     "required": True,  # sets "required input" marker
#     "description": [
#         "So it is, and so it goes, see lnk_to_param(here|param1) and lnk_to_param(`param2=4`|param2) and lnk_to_param(param3).",
#     ],
#     "required_if": " depends on ...",
#
#     "type": "[array of ][`integer`,`real`,`string`][of any size, of size x]"  ,
#     "allowed":'', #either write here what is allowed, like >0, or set the allowed values in the allowed_table
#     "allowed_table": [
#         ("`0`", "first option"),
#         ("`1`", "second option, see lnk_to_param(param4)"),
#     ],
#     "default":"`default value` or how the default depends on other variables",
# }


# X1_a_cos(0,n)	$X^1$ cosine mode coefficients of the axis initial guess, $\cos(-n n_{FP}\zeta)$, toroidal n mode number		WhichInitEquilibrium=0
# X1_a_sin(0,n)	$X^1$ sine mode coefficients of the axis initial guess, $\sin(-n n_{FP}\zeta)$, toroidal n mode number		WhichInitEquilibrium=0
# X2_a_cos(0,n)	$X^2$ cosine mode coefficients of the axis initial guess, $\cos(-n n_{FP}\zeta)$, toroidal n mode number		WhichInitEquilibrium=0
# X2_a_sin(0,n)	$X^2$ sine mode coefficients of the axis initial guess, $\sin(-n n_{FP}\zeta)$, toroidal n mode number		WhichInitEquilibrium=0
# X1pert_b_cos(m,n)	Add perturbation on the boundary in $X^1$ cosine mode coefficient,  $\cos(m\vartheta-n n_{FP}\zeta)$		boundary_perturb=T
# X1pert_b_sin(m,n)	Add perturbation on the boundary in $X^1$ sine mode coefficient,  $\sin(m\vartheta-n n_{FP}\zeta)$		boundary_perturb=T
# X2pert_b_cos(m,n)	Add perturbation on the boundary in $X^2$ cosine mode coefficient,  $\cos(m\vartheta-n n_{FP}\zeta)$		boundary_perturb=T
# X2pert_b_sin(m,n)	Add perturbation on the boundary in $X^2$ sine mode coefficient,  $\sin(m\vartheta-n n_{FP}\zeta)$		boundary_perturb=T


dict["ProjectName"] = {
    "category": ["Initialization"],
    # "subtitle":
    # "required": True,
    "description": [
        "Project name, used for all output files, must be a string without spaces",
    ],
    # "required_if": "",
    "default": "`GVEC`",
    "type": "`string`",
    "allowed": "",
}

dict["whichInitEquilibrium"] = {
    "category": ["Initialization"],
    "subtitle": "mandatory",
    "required": True,
    "description": [
        "How initial guess is computed. Either from boundary and axis parameters, or from VMEC file.",
    ],
    # "required_if": "",
    # "default":"",
    "type": "`integer`",
    # "allowed": '',
    "allowed_table": [
        ("`0`", "from axis and boundary parameters"),
        ("`1`", "from VMEC file, **needs:** lnk_to_param(VMECwoutfile)"),
    ],
}

dict["nfp"] = {
    "category": ["Discretization"],
    "subtitle": "mandatory parameter",
    "required": True,
    "description": [
        "This parameter sets the number of field periods $N_{FP}$",
    ],
    "type": "`integer`",
    # "required_if": "",
    "allowed": "$> 0$",
}

for var, varltx in zip(["X1", "X2", "LA"], [r"$X^1$", r"$X^2$", r"$\lambda$"]):
    dict[f"{var}_mn_max"] = {
        "category": ["Discretization"],
        "subtitle": "mandatory if `whichInitEquilibrium=0`",
        # "required": True,
        "description": [
            "The solution is described by three variables "
            + r"$X^1,X^2,\lambda$ with a B-Spline discretization in radial direction$\rho$ and a double-periodic Fourier series in the poloidal angle $\vartheta$ and the toroidal angle $\zeta$.",
            r"This parameter sets maximum poloidal and toroidal mode numbers $m_\text{max}$,$n_\text{max}$ of the variable"
            + varltx,
            "Overwrites the default, which is only set if lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium) (initialize with VMEC file).",
        ],
        "type": "array of `integer` of size 2",
        "required_if": "lnk_to_param(`whichInitEquilibrium=0`|whichInitEquilibrium)",
        "default": "only if lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium), the default is set to the maximum mode numbers from the VMEC solution file.",
        "allowed": r"$m_\text{max}\gt 0,n_\text{max}\ge 0$",
    }

for var, varltx, dflt in zip(
    ["X1", "X2", "LA"],
    [r"$X^1$", r"$X^2$", r"$\lambda$"],
    ["`_cos_`", "`_sin_`", "`_sin_`"],
):
    dict[f"{var}_sin_cos"] = {
        "category": ["Discretization"],
        # "subtitle": ,
        # "required": True,
        "description": [
            "The solution is described by three variables "
            + r"$X^1,X^2,\lambda$ with a B-Spline discretization in radial direction$\rho$ and a double-periodic Fourier series in the poloidal angle $\vartheta$ and the toroidal angle $\zeta$.",
            r"This parameter sets which Fourier modes are active for the variable"
            + varltx
            + ", thus allows to impose stellarator symmetry.",
        ],
        "type": "`string`",
        # "required_if": "lnk_to_param(`whichInitEquilibrium=0`|whichInitEquilibrium)",
        "default": dflt
        + " if lnk_to_param(`whichInitEquilibrium=0`|whichInitEquilibrium). "
        + "Ff lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium), the default is set from the VMEC solution.",
        "allowed_table": [
            (
                "`_sin_`",
                r"only use $\sin(m\vartheta-n\zeta)$, with $\{m=0,1\leq n\leq n_\text{max}\,\, 1\leq m\leq m_\text{max},-n_\text{max}\leq n\leq n_\text{max}\}$",
            ),
            (
                "`_cos_`",
                r"only use $\cos(m\vartheta-n\zeta)$, with $\{m=0,0\leq n\leq n_\text{max}\,; \,1\leq m\leq m_\text{max},-n_\text{max}\leq n\leq n_\text{max}\}$",
            ),
            ("`_sincos_`", "use both sine and cosine modes"),
        ],
    }
for var, varltx in zip(
    ["X1X2", "LA"],
    [r"variables $X^1,X^2$", r"variable $\lambda$"],
):
    dict[f"{var}_deg"] = {
        "category": ["Discretization"],
        "subtitle": "mandatory parameter",
        "required": True,
        "description": [
            "The solution is described by three variables "
            + r"$X^1,X^2,\lambda$ with a B-Spline discretization in radial direction$\rho$ and a double-periodic Fourier series in the poloidal angle $\vartheta$ and the toroidal angle $\zeta$.",
            r"This parameter sets the polynomial degree $p$ of the B-Spline, for the "
            + varltx
            + ".",
        ],
        "type": "`integer`",
        # "required_if": "",
        # "default":"",
        "allowed": "$>0$",
    }
    dict[f"{var}_continuity"] = {
        "category": ["Discretization"],
        # "subtitle": "mandatory parameter",
        # "required": True,
        "description": [
            "The solution is described by three variables "
            + r"$X^1,X^2,\lambda$ with a B-Spline discretization in radial direction$\rho$ and a double-periodic Fourier series in the poloidal angle $\vartheta$ and the toroidal angle $\zeta$.",
            r"This parameter sets the continuity of the B-Spline, for the "
            + varltx
            + ".",
            f"For now, only a continuity of  $p-1$  is possible, with $p$ from lnk_to_param({var}_deg).",
        ],
        "type": "`integer`",
        # "required_if": "",
        "default": f"lnk_to_param(`{var}_deg-1`|{var}_deg)",
        "allowed": "$\geq 0$",
    }


## boundary


# Initialization, boundary	getBoundaryFromFile	Read the boundary data from a specific netcdf file, where $X^1,X^2$ are given pointwise over equidistant $\vartheta,\zeta$ grid, on one field period. The boundary will be projected to the Fourier representation defined by `X1_mn_max,X1_sin_cos,X2_mn_max,X2_sincos`	{-1: off, 1: read from specific netcdf file}		-1	INTEGER
# Initialization, boundary	boundary_filename	Full file name of the boundary data file.		GetBoundaryFromFile=1	must be provided	GETSTR


for xd in ["1", "2"]:
    for cs, sc in zip(["cos", "sin"], ["cos", "sin"]):
        dict[f"X{xd}_b_{cs}(m,n)"] = {
            "category": ["boundary"],
            # "subtitle":
            # "required": True,
            "description": [
                "The boundary is described by two variables "
                + r"$X^1(\vartheta,\zeta)$,$X^2(\vartheta,\zeta)$, as a double-periodic Fourier series.",
                f"This parameter sets the coefficient of the Fourier mode of $X^{xd}$ variable, with basis function $\{cs}"
                + r"(m\vartheta-n N_{FP}\zeta)$. The number of field periods lnk_to_param($N_{FP}$|nfp).",
                "`m,n` must be replaced by the poloidal ($m$) and toroidal ($n$) mode number (excluding the $N_{FP}$ factor).",
                r"The maximum mode number $m_\text{max}$,$n_\text{max}$ is provided by "
                + f"lnk_to_param(X{xd}_mn_max). ",
                f"ignored if lnk_to_param(`X{xd}_sin_cos=_{sc}_`|X{xd}_sin_cos)",
            ],
            "required_if": "lnk_to_param(`whichInitEquilibrium`=0|whichInitEquilibrium).",
            "default": "`0`",
            "type": "`real`",
            "allowed": r"Coefficients are read for mode numbers $\{m=0,0\le n\le n_\text{max}\}$ and $\{1\le m\le m_\text{max},-n_\text{max}\le n\le n_\text{max}\}$",
        }

dict["VMECwoutfile"] = {
    "category": ["Initialization", "VMEC"],
    # "subtitle":
    "description": [
        "full file name of vmec solution file, either as netcdf or as nemec output,",
        "**needs** lnk_to_param(VMECwoutfile_format)",
    ],
    "required_if": "lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium)",
    "type": "`string`",
    # "allowed": '',
    # "default":"",
}
dict["VMECwoutfile_format"] = {
    "category": ["Initialization", "VMEC"],
    # "subtitle":
    "description": [
        "Choose which VMEC wout file format, either netcdf or ascii/binary format from NEMEC",
        "The filename is specified in lnk_to_param(VMECwoutfile)",
    ],
    "required_if": "lnk_to_param(`whichInitEquilibrium=1`|whichInitEquilibrium)",
    "type": "`string`",
    "allowed_table": [
        ("`0`", "netcdf format"),
        ("`1`", "ascii format from older vmec (nemec)"),
        ("`2`", "binary format from older vmec (nemec)"),
    ],
    "default": "`0`",
}

# profiles

# profile input	PHIEDGE	Total toroidal flux in Wb, including the $2\pi$ factor.			1.0	REAL

dict["PhiEdge"] = {
    "category": ["profiles"],
    "description": [
        r"Total toroidal magnetic flux $\Phi$ at the boundary. Defines the $\Phi$ profile. A positive sign means that the toroidal magnetic field points in positive $\zeta$ direction.",
        r"Note that internally, GVEC then uses $\Phi_\text{edge}/(2\pi)$.",
    ],
    "type": "`real`",
    "default": "`1.0`",
}
for prof, proflong in zip(["iota", "pres"], ["rotational transform", "pressure"]):
    dict[f"{prof}_type"] = {
        "category": ["profiles"],
        "subtitle": "mandatory if `whichInitEquilibrium=0`",
        "required_if": "`whichInitEquilibrium=0`",
        "description": [
            f"Type of profile to represent the {proflong} `{prof}(s)`. Can be a polynomial, B-Spline or a set of points which is then interpolated",
            "**Note:** the profile is always given as a function of the normalized magnetic flux $s$, with $s=0$ as the magnetic axis, and $s=1$ at the boundary. The normalized radius-like coordinate is "
            + r"$\rho=\sqrt{s}$.",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                "`polynomial`",
                f"polynomial represenation, **needs** lnk_to_param({prof}_coefs)",
            ),
            (
                "`bspline`",
                f"B-Spline representation, **needs** lnk_to_param({prof}_coefs) and lnk_to_param({prof}_knots)",
            ),
            (
                "`interpolation`",
                f"Cubic spline interpolation from point values at s-positions, **needs** lnk_to_param({prof}_rho2) and lnk_to_param({prof}_vals)",
            ),
        ],
    }
    dict[f"{prof}_coefs"] = {
        "category": ["profiles"],
        # "subtitle": ,
        "description": [
            f"Depending on the type of profile of the {proflong} `{prof}(s)`:",
            f"- if lnk_to_param(`{prof}_type=polynomial`|{prof}_type), it sets the coefficients $c_0 + c_1 s + c_2 s^2 \dots$",
            f"- if lnk_to_param(`{prof}_type=bspline`|{prof}_type), these are the B-Spline coefficients (which need to be compatible to the knots lnk_to_param({prof}_knots))",
        ],
        "type": "array of `real` of any size",
        "required_if": f"lnk_to_param({prof}_type) is `polynomial` or `bspline`",
    }
    dict[f"{prof}_knots"] = {
        "category": ["profiles"],
        # "subtitle":
        "description": [
            f"Required for the B-Spline profile of the {proflong} `{prof}(s)`:",
            "The knots must be in the range of $s=[0,1]$ and must be monotonically increasing.",
            "They must have the same multiplicity of the first knot at $s=0$ and the last knot at $s=1$. The degree of the B-Spline is deduced from the multiplicity.",
            f"Knots need to be compatible with the B-Spline coefficients lnk_to_param({prof}_coefs)",
        ],
        "type": "array of `real` of any size",
        "required_if": f"lnk_to_param({prof}_type) is `bspline`",
    }
    dict[f"{prof}_rho2"] = {
        "category": ["profiles"],
        # "subtitle":
        "description": [
            f"Required for the cubic spline interpolation of the {proflong} `{prof}(s)`:",
            "These are the radial point positions, in the normalized magnetic flux",
            "They must cover the range of $s=[0,1]$ and must be monotonically increasing.",
            f"Point positions must of same size as the values, specified by lnk_to_param({prof}_vals)",
        ],
        "type": "array of `real` of any size",
        "required_if": f"lnk_to_param({prof}_type) is `interpolation`",
    }
    dict[f"{prof}_vals"] = {
        "category": ["profiles"],
        # "subtitle": True
        "description": [
            f"Required for the cubic spline interpolation of the {proflong} `{prof}(s)`:",
            f"These are the values of `{prof}` at the radial point positions, in the normalized magnetic flux",
            f"Point positions must of same size as the values, specified by lnk_to_param({prof}_rho2)",
        ],
        "type": "array of `real` of any size",
        "required_if": f"lnk_to_param({prof}_type) is `interpolation`",
    }
    dict[f"{prof}_BC_type_axis"] = {
        "category": ["profiles"],
        # "subtitle": True
        "description": [
            f"Boundary condition at $s=0$ for the cubic spline interpolation of the {proflong} `{prof}(s)`",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                '`"not_a_knot"`',
                "Makes the third derivative continuous at the second grid point",
            ),
            (
                '`"1st_deriv"`',
                f"Sets the first derivative at the boundary $s=0$, either to zero or to the provided first value of lnk_to_param({prof}_BC_vals)",
            ),
            (
                '`"2nd_deriv"`',
                f"Sets the second derivative at the boundary $s=0$, either to zero or to the provided first value of lnk_to_param({prof}_BC_vals)",
            ),
        ],
        "required_if": f"lnk_to_param({prof}_type) is `interpolation`",
        "default": '`"not_a_knot"`',
    }
    dict[f"{prof}_BC_type_edge"] = {
        "category": ["profiles"],
        # "subtitle": True
        "description": [
            f"Boundary condition at $s=1$ for the cubic spline interpolation of the {proflong} `{prof}(s)`",
        ],
        "type": "`string`",
        "allowed_table": [
            (
                '`"not_a_knot"`',
                "Makes the third derivative continuous at the second to last grid point",
            ),
            (
                '`"1st_deriv"`',
                f"Sets the first derivative at the boundary $s=1$, either to zero or to the provided second value of lnk_to_param({prof}_BC_vals)",
            ),
            (
                '`"2nd_deriv"`',
                f"Sets the second derivative at the boundary $s=0$, either to zero or to the provided second value of lnk_to_param({prof}_BC_vals)",
            ),
        ],
        "required_if": f"lnk_to_param({prof}_type) is `interpolation`",
        "default": '`"not_a_knot"`',
    }
    dict[f"{prof}_BC_vals"] = {
        "category": ["profiles"],
        # "subtitle": True
        "description": [
            f"Values for the first / second derivative, at $s=0$ and $s=1$, for the cubic spline interpolation of the {proflong} `{prof}(s)`.",
        ],
        "type": "array of `real` of size 2",
        "required_if": f'lnk_to_param({prof}_type) is `interpolation` , and if lnk_to_param({prof}_BC_type_axis) or lnk_to_param({prof}_BC_type_edge) is different from `"not_a_knot"`',
        "default": "`(/0.0,0.0/))`",
    }
    dict[f"{prof}_scale"] = {
        "category": ["profiles"],
        # "subtitle": "optional"
        "description": [
            "Scales the {proflong} `{prof}(s)` by a constant.",
            f"See profile definition lnk_to_param({prof}_type)",
        ],
        "type": "`real`",
        "default": "`1.0`",
    }


with open("parameters.yaml", "w") as f:
    yaml.safe_dump(dict, f, allow_unicode=True, sort_keys=False)

# with open("parameters.yaml", "r") as f:
#    dict_check = yaml.safe_load(f)

# print(dict.keys())
# print(dict_check.keys())

In [ ]:
from generate_parameter_list import format_parameter_list

format_parameter_list(
    "parameters.yaml",
    # output_file="parameters-all.md",
    formatting="screen",
    filter_expr="'X1' in 'name' and 'Init' in 'category' ",
)